<h4>Spark运行时架构</h4>

- Spark集群采用的主/从结构，
- 驱动器(Driver)节点调度各个分布式工作节点
- 工作节点又被称为执行器(executor)节点，是一种工作进程，在Spark作业中运行任务
- 驱动器节点和所有执行器节点合称为一个Spark应用
- Spark应用通过集群管理器(Cluster Manager)的外部服务在集群的机器上启动
- Spark自带独立集群管理器，也能运行在Hadoop Yarn和Apache Mesos上

<h4>驱动器节点</h4>

- Spark驱动器是执行程序中main()方法的进程，执行用户编写的SparkContext、创建RDD、以及RDD的转化和行动操作。
- 在启动Spark Shell时就启动了Spark驱动器，并且Spark shell会预先加载一个教作sc的SparkContext对象

- 把用户程序转为任务
    - 从输入数据，到创建RDD，到转化操作派生新的RDD，最后行动操作，Spark会将这些操作隐式转换成有向无环图（Directed Acyclic Graph）
    - 当驱动器运行时，它会把DAG变为多个物理执行单元，这些单元也称为任务
    
- 为执行器节点调度任务
    - Spark驱动器程序可以在个执行器进程间协调任务的调度
    - 驱动器程序会将Spark运行时的信息通过http://localhost:4040上展示出来

<h4>执行器节点</h4>

- 一种工作进程，Spark应用启动时，执行器节点同时启动
- 运行组成Spark应用的任务，将结果返回给驱动器进程
- 通过自身块管理器(Block Manager)为用户程序中要求缓存的RDD提供内存式存储，让RDD直接缓存在执行器进程内

<b>本地模式下，Spark驱动器和执行器程序在同一个Java进程中运行</b>

集群运行Spark应用全过程

1. 用户通过spark-submit脚本提交应用，bin/spark-submit my_script.py
2. spark-submit启动驱动器程序，调用用户定义的main()方法
3. 驱动器程序与集群管理器通信，申请资源以启动执行器节点
4. 集群管理器为驱动器启动执行器节点
5. 驱动器进程执行用户应用中的操作。根据程序中定义的对RDD的转化和行动操作，驱动器把工作以任务的形式发送到执行器进程
6. 任务在执行器中进行计算并保存结果
7. 如果驱动器程序的main()方法退出，或调用了SparkContext.stop()，驱动器会终止执行器进程，并通过集群管理器释放资源

写一个分发文件用的脚本

192.168.0.200   zxtest<br>
192.168.0.201   hadoop-2<br>
192.168.0.202   hadoop-3<br>
192.168.0.203   hadoop-4

In [1]:
%%html
<img src="img/xsync.png", width=400>

<h4>启动独立集群管理器</h4>

1. 使用xsync将spark传至其余三个虚拟机中
2. 编辑conf/slaves，将我们的ip地址和名称映射写进去
3. 在主节点上运行sbin/start-all.sh
4. 停止集群，在主节点上运行sbin/stop-all.sh
5. 通过 <span>http://192.168.0.200:8080</span> 查看集群管理器的网页用户界面
6. 可以看到主节点的url是 spark://zxtest:7077

In [2]:
%%html
<img src="img/admin.png", width=800>

<h4>spark-submit的--master标记可以接收的值</h4>

|值|描述|
|:----|:----|
|spark://host:port|连接到指定端口的Spark独立集群上，默认情况下Spark独立主节点使用7077端口
|mesos://host:port|连接到指定端口的Mesos集群上，默认情况下Mesos主节点监听5050端口
|yarn|连接到一个YARN集群，需要设置变量HADOOP_CONF_DIR指向Hadoop配置目录，以获取集群信息
|local|运行本地模式，使用单核
|local[N]|运行本地模式，使用N个核心
|local[\*]|运行本地模式，使用尽可能多的核心

<h4>spark-submit的一般格式</h4>

- bin/spark-submit [options] <app jar | python file> [app options]
- [option]是传给spark-submit的标记列表，可以运行spark-submit --help列出所有可接收标记
- [app options]是传给应用的选项

|标记|描述|
|:----|:----|
|--master|表示要连接的集群管理器
|--deploy-mode|选择在本地客户端("client")，即调用spark-submit的机器上启动驱动器程序，还是集群中的一台工作节点上("cluster")启动，默认为本地模式
|--class|运行Java或Scala程序时应用的主类
|--name|应用的显示名，会显示在Spark的网页用户界面中
|--files|需要放到应用工作目录中的文件列表
|--py-files|需要放到PYTHONPATH中的文件列表
|--execute-memory|执行器进程使用的内存量，以字节为单位，如"512m"(512MB)或"15g"(15GB)
|--driver-memory|驱动器进程使用的内存量

<h3>以下用不同方法尝试启动wordcount.py</h3>

<h4>本地提交计算</h4>

./bin/spark-submit examples/src/main/python/wordcount.py file:///opt/spark/data/test/a.txt

In [3]:
%%html
<img src="img/localsubmit.png", width=600>

<h4>Spark独立集群管理器提交计算</h4>

- 首先编辑主节点的conf/slaves文件并填上所有工作节点的主机名
- sbin/start-all.sh启动
- ./bin/spark-submit --master spark://192.168.0.200:7077 examples/src/main/python/wordcount.py file:///opt/spark/data/test/a.txt
- http://192.168.0.200:8080 查看集群状态
- 可以使用 --master参数以同样的方式启动spark-shell或pyspark，来连接到该集群上
    - spark-shell --master spark://masternode:7077
    - pyspark --master spark://masternode:7077
- --executor-memory标记传递的值，超过了集群所能提供的内存总量，那么独立集群管理器始终无法为应用分配执行器节点


<h4>独立集群管理器支持两种部署模式</h4>

1. 客户端模式
    - 驱动器程序会运行在执行spark-submit的机器上，要求提交应用的机器与工作节点间有很快的网络速度
2. 集群模式
    - 驱动器程序会作为某个工作节点上独立的一个进程运行在独立集群管理器内部，也会连接主节点来申请执行器节点
    - --deploy-mode cluster
    

<h4>配置资源用量</h4>

1. 执行器进程内存
    - 你可以通过spark-submit的--executor-memory参数来配置此项，控制执行器节点占用工作节点多少内存
2. 占用核心总数的最大值
    - --total-executorcores限制一个应用的总核心数用量

In [4]:
%%html
<img src="img/standalone.png", width=600>

<h4>Yarn集群提交计算</h4>

- HADOOP_CONF_DIR配置环境变量
- 关闭spark集群，启动hadoop集群
- ./bin/spark-submit --master yarn examples/src/main/python/wordcount.py file:///opt/spark/data/test/a.txt

- Yarn是Hadoop2.0引入的集群管理器，在Yarn上配置Spark可以让Spark在存储数据的物理节点上运行
- --executor-cores设置每个执行器进程从YARN中占用的核心数目

In [5]:
%%html
<img src="img/yarn.png", width=600>